## Import the libraries

In [245]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Import the dataset

In [246]:
dataset_filepath = '/Users/davronabdukhakimov/Desktop/RealEstate_California.csv'
raw_dataset = pd.read_csv(dataset_filepath)

raw_dataset['date_posted'] = pd.to_datetime(raw_dataset['datePostedString'],format='%Y-%m-%d')
raw_dataset.dtypes

Unnamed: 0                     int64
id                            object
stateId                        int64
countyId                       int64
cityId                         int64
country                       object
datePostedString              object
is_bankOwned                   int64
is_forAuction                  int64
event                         object
time                         float64
price                        float64
pricePerSquareFoot           float64
city                          object
state                         object
yearBuilt                      int64
streetAddress                 object
zipcode                      float64
longitude                    float64
latitude                     float64
hasBadGeocode                  int64
description                   object
currency                      object
livingArea                   float64
livingAreaValue              float64
lotAreaUnits                  object
bathrooms                    float64
b

## Dealing with missing values

### Drop unnecessary columns

In [247]:
columns_to_drop = ['Unnamed: 0','cityId','id','stateId','currency','countyId','state','country','description','time','datePostedString','hasBadGeocode','is_bankOwned','is_forAuction','lotAreaUnits']
clean_dataset = raw_dataset.drop(columns_to_drop,axis = 1)

### Drop rows with nan values, duplicates and rows with event type of rent

In [248]:
clean_dataset.dropna(inplace=True)
clean_dataset.drop_duplicates(inplace=True)
clean_dataset = clean_dataset[clean_dataset['event'] != 'Listed for rent']
clean_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31049 entries, 0 to 35388
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   event               31049 non-null  object        
 1   price               31049 non-null  float64       
 2   pricePerSquareFoot  31049 non-null  float64       
 3   city                31049 non-null  object        
 4   yearBuilt           31049 non-null  int64         
 5   streetAddress       31049 non-null  object        
 6   zipcode             31049 non-null  float64       
 7   longitude           31049 non-null  float64       
 8   latitude            31049 non-null  float64       
 9   livingArea          31049 non-null  float64       
 10  livingAreaValue     31049 non-null  float64       
 11  bathrooms           31049 non-null  float64       
 12  bedrooms            31049 non-null  float64       
 13  buildingArea        31049 non-null  float64       


### Extract season from date_posted column

In [249]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'
    
clean_dataset['season_posted'] = clean_dataset['date_posted'].apply(get_season)
clean_dataset.drop('date_posted',axis=1,inplace=True)
clean_dataset.head()

,event,price,pricePerSquareFoot,city,yearBuilt,streetAddress,zipcode,longitude,latitude,livingArea,...,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county,season_posted
0,Listed for sale,145000.0,0.0,Gold Run,0,0 Moody Ridge Rd,95717.0,-120.834473,39.167866,0.0,...,0.0,0,0,0,0,0,0,LOT,Placer County,winter
1,Listed for sale,675000.0,404.0,Pinole,1958,1476 Belden Ct,94564.0,-122.300613,38.001213,1671.0,...,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer
2,Listed for sale,649000.0,459.0,Pinole,1959,3540 Savage Ave,94564.0,-122.271362,37.982040,1414.0,...,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer
3,Listed for sale,599000.0,448.0,Pinole,1908,2391 Plum St,94564.0,-122.289230,38.004395,1336.0,...,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County,summer
4,Listed for sale,299000.0,0.0,Pinole,0,2693 Appian Way,94564.0,-122.298439,37.986309,0.0,...,0.0,0,0,0,0,0,0,LOT,Contra Costa County,summer


### Extract street name from streetAddress column

In [250]:
split_address = clean_dataset['streetAddress'].str.split(' ',n=1,expand=True)
clean_dataset = clean_dataset[split_address[1].notna()]
clean_dataset = clean_dataset[split_address[0].str.isdigit()]
clean_dataset['street_name'] = split_address[1]
clean_dataset.drop('streetAddress',axis = 1,inplace=True)

/var/folders/r1/m92fhd0n5mq0_bh6sg3_pnlh0000gn/T/ipykernel_69887/2322718560.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_dataset = clean_dataset[split_address[0].str.isdigit()]


### Drop livingArea since the column contains the same values as livingAreaValue column

In [251]:
clean_dataset[clean_dataset['livingArea'] != clean_dataset['livingAreaValue']]
clean_dataset.drop('livingArea',axis=1,inplace=True)

### Drop hasGarage column since garageSpaces column is more informative

In [252]:
clean_dataset.drop('hasGarage',axis=1,inplace=True)

### Split levels column into 2 columns

In [253]:
zero_level = ['0']
one_level = ['One','One Story','One Story-One','One-Two','1','One-Multi/Split','Other-One','One-Three Or More',
             'One Story-Two','One Story-Three Or More','Multi/Split-One','One-Two-Three Or More','One-Two-Multi/Split']
two_level = ['Two','Two Story','Two Story-Two','2','Two-Multi/Split','Multi/Split-Two','Two-Three Or More','Two Story-One',
             'Two Story-Three Or More','Two-Three Or More-Multi/Split','Two-One']
three_level_more = ['Three Or More','Three or More Stories','Tri-Level','Three or More Stories-Three Or More',
                    'Three Or More-Multi/Split','3','Three','Three or More Stories-One','Three or More Stories-Two',
                   'Four','Tri-Level-Three Or More','Multi/Split-Three Or More','Multi/Split-Tri-Level','4+','Five or More',
                    'Three Or More-Split Level','Tri-Level-Two','Three Or More-Two','Three or More Stories-One-Two']


total = zero_level
total.extend(one_level)
total.extend(two_level)
total.extend(three_level_more)
missing_type = [val for val in clean_dataset['levels'].unique() if val not in total]

def get_level(level):
    if level in one_level:
        if 'split' in level.lower() or 'multi' in level.lower():
            return ['one', 1]
        else:
            return ['one', 0]
    elif level in two_level:
        if 'split' in level.lower() or 'multi' in level.lower():
            return ['two', 1]
        else:
            return ['two', 0]
    elif level in three_level_more:
        if 'split' in level.lower() or 'multi' in level.lower():
            return ['three+', 1]
        else:
            return ['three+', 0]
    else:
        if 'split' in level.lower() or 'multi' in level.lower():
            return ['zero', 1]
        else:
            return ['zero', 0]

clean_dataset[['level','multi/split']] = clean_dataset['levels'].apply(lambda x: pd.Series(get_level(x)))
clean_dataset.drop('levels',axis=1,inplace=True)

### Drop rows with homeType of LOT since we don't have their area (can be extracted from description column)

In [254]:
clean_dataset = clean_dataset[clean_dataset['homeType'] != 'LOT']
clean_dataset

,event,price,pricePerSquareFoot,city,yearBuilt,zipcode,longitude,latitude,livingAreaValue,bathrooms,...,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county,season_posted,street_name,level,multi/split
1,Listed for sale,675000.0,404.0,Pinole,1958,94564.0,-122.300613,38.001213,1671.0,2.0,...,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer,Belden Ct,one,0
2,Listed for sale,649000.0,459.0,Pinole,1959,94564.0,-122.271362,37.982040,1414.0,2.0,...,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer,Savage Ave,one,0
3,Listed for sale,599000.0,448.0,Pinole,1908,94564.0,-122.289230,38.004395,1336.0,2.0,...,0,1,0,0,SINGLE_FAMILY,Contra Costa County,summer,Plum St,two,0
5,Listed for sale,575000.0,407.0,Pinole,1958,94564.0,-122.257256,37.982376,1413.0,2.0,...,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer,Doidge Ave,one,0
6,Listed for sale,888000.0,390.0,Pinole,2000,94564.0,-122.309303,38.008163,2278.0,3.0,...,0,0,0,0,SINGLE_FAMILY,Contra Costa County,summer,Carlotta Cir,two,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35382,Listed for sale,2249000.0,0.0,Los Angeles,1925,90036.0,-118.357658,34.079071,0.0,0.0,...,0,0,0,0,MULTI_FAMILY,Los Angeles County,summer,N Spaulding Ave,zero,0
35383,Listed for sale,2295000.0,0.0,Los Angeles,1924,90036.0,-118.352928,34.082546,0.0,0.0,...,0,0,0,0,MULTI_FAMILY,Los Angeles County,summer,N Gardner St,zero,0
35384,Listing removed,3360000.0,0.0,Los Angeles,1927,90036.0,-118.346329,34.061150,0.0,4.0,...,0,0,0,0,SINGLE_FAMILY,Los Angeles County,summer,S Cloverdale Ave,zero,0
35385,Listing removed,1699000.0,958.0,Los Angeles,1924,90036.0,-118.354790,34.058182,1774.0,2.0,...,0,0,0,0,SINGLE_FAMILY,Los Angeles County,summer,Masselin Ave,zero,0


## Data analysis

### Ordinal encoding categorical columns

In [255]:
from sklearn.preprocessing import OrdinalEncoder

cat_columns = clean_dataset.select_dtypes(include = 'object').columns

ordinal_encoder = OrdinalEncoder()
clean_dataset[cat_columns] = ordinal_encoder.fit_transform(clean_dataset[cat_columns])

In [256]:
correlations = clean_dataset.corr()
correlations = correlations.round(3)
correlations.iloc[:,:13]

,event,price,pricePerSquareFoot,city,yearBuilt,zipcode,longitude,latitude,livingAreaValue,bathrooms,bedrooms,buildingArea,parking
event,1.000,-0.011,-0.007,-0.014,-0.008,-0.055,0.067,-0.055,-0.003,0.019,-0.001,-0.004,0.010
price,-0.011,1.000,0.011,0.001,-0.051,-0.083,-0.035,-0.044,0.035,0.452,0.215,0.025,-0.075
pricePerSquareFoot,-0.007,0.011,1.000,-0.011,0.031,-0.006,0.004,-0.007,-0.001,-0.021,-0.028,-0.000,-0.004
city,-0.014,0.001,-0.011,1.000,-0.006,0.063,-0.071,0.036,-0.010,-0.012,-0.010,-0.011,-0.030
yearBuilt,-0.008,-0.051,0.031,-0.006,1.000,-0.065,0.102,-0.105,-0.053,0.040,0.027,-0.054,0.071
zipcode,-0.055,-0.083,-0.006,0.063,-0.065,1.000,-0.787,0.881,0.001,0.001,0.073,0.003,-0.031
longitude,0.067,-0.035,0.004,-0.071,0.102,-0.787,1.000,-0.912,-0.002,0.015,0.001,-0.008,0.149
latitude,-0.055,-0.044,-0.007,0.036,-0.105,0.881,-0.912,1.000,-0.003,-0.046,0.008,0.001,-0.115
livingAreaValue,-0.003,0.035,-0.001,-0.010,-0.053,0.001,-0.002,-0.003,1.000,0.012,0.014,0.999,-0.011
bathrooms,0.019,0.452,-0.021,-0.012,0.040,0.001,0.015,-0.046,0.012,1.000,0.673,-0.006,0.111


In [257]:
correlations.iloc[:,13:]

,garageSpaces,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county,season_posted,street_name,level,multi/split
event,0.036,-0.011,0.010,0.015,-0.008,-0.009,-0.003,-0.170,0.005,-0.035,0.020
price,0.117,0.178,0.124,0.043,0.008,0.046,0.007,-0.071,0.006,0.101,0.028
pricePerSquareFoot,0.022,0.001,-0.001,-0.001,-0.002,-0.013,0.004,0.004,-0.005,0.005,-0.002
city,-0.052,-0.013,0.003,0.009,0.038,-0.030,0.226,0.002,-0.022,0.064,-0.010
yearBuilt,0.066,0.040,0.054,-0.048,-0.024,0.001,-0.014,0.041,0.006,-0.058,0.006
zipcode,-0.282,-0.012,-0.138,0.007,0.056,0.124,0.211,-0.032,-0.039,0.333,-0.049
longitude,0.314,0.048,0.247,-0.019,-0.106,-0.030,-0.030,0.064,0.029,-0.371,0.013
latitude,-0.344,-0.030,-0.259,0.010,0.056,0.086,0.027,-0.073,-0.021,0.393,-0.037
livingAreaValue,0.002,0.005,0.001,0.001,-0.002,0.007,0.007,0.018,0.007,0.011,0.000
bathrooms,0.186,0.264,0.229,0.067,-0.005,0.218,0.051,-0.007,0.003,0.102,0.056
